In [ ]:
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torchmetrics import AveragePrecision
from torchmetrics import Recall
import numpy as np
import matplotlib.pyplot as plt
from data import get_dataloaders

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = 'results/results_resnet18_4_001.pkl'

with open(path, 'rb') as handle:
    data = pickle.load(handle)

In [ ]:
plt.title(f'Accuracy against Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(data['train_acc'], label = 'train')
plt.plot(data['val_acc'], label = 'validation')
plt.legend()

# Loss v.s. Epoch

# Performance on Test Set

Test loss and accuracy:

In [ ]:
device = torch.device("cuda:{}".format(0) if torch.cuda.is_available() else 'cpu')
dataloader = get_dataloaders('data/train_resized.hdf5', 'data/study_label.csv')
model = models.resnet18(num_classes=4)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.load_state_dict(torch.load("models/results_resnet18_4_001.pt", map_location=device))
model = model.to(device)
criterion = nn.CrossEntropyLoss()

Test recall and mean precision: